In [ ]:
# # Test LST-bin run

# %run /lustre/aoc/projects/hera/mmolnar/LST_bin/hera_cal/scripts/lstbin_run.py \
# --dlst 0.001565970008 --file_ext grp1.of1.{type}.{time:7.5f}.HH.OCRSL.uvh5 \
# --outdir /lustre/aoc/projects/hera/mmolnar/LST_bin/binned_files \
# --ntimes_per_file 60 --rephase --sig_clip --sigma 4 --output_file_select 0 \
# --min_N 0 --lst_start 1.2 --vis_units Jy \
# --Nbls_to_load 150 --fixed_lst_start --input_cals \
# '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458098/zen.2458098.*.HH.smooth_abs.calfits' \
# '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458099/zen.2458099.*.HH.smooth_abs.calfits' \
# --overwrite \
# '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458098/zen.2458098.*.HH.uvh5' \
# '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458099/zen.2458099.*.HH.uvh5'

## Reading MAD-Clip flags

In [ ]:
import lzma
import os
import pickle
import sys
from collections import OrderedDict as odict

import pandas as pd

In [ ]:
sys.path.insert(1, '/lustre/aoc/projects/hera/mmolnar/simpleredcal')

from align_utils import idr2_jdsx

In [ ]:
clipped_flags = '/lustre/aoc/projects/hera/mmolnar/LST_bin/binned_files/zen.grp1.of1.LST.1.31552.bad_jds.pkl'
with open(clipped_flags, "rb") as f:
    clip_f = pickle.load(f)

In [ ]:
# Creating a odict of flags
clip_f_copy = {k: v for k, v in clip_f.items()}
new_flags = odict()
for k, v in clip_f_copy.items():
    bl_flags = odict()
    for t, f in v.items():
        # De-bin data
        bl_flags[t*2] = f[::2]
        bl_flags[t*2 + 1] = f[1::2]
    new_flags[k] = bl_flags

In [ ]:
new_flags_fn = clipped_flags.replace('zen.grp1.of1.LST', 'clipped_flags').replace('bad_jds.', '')

if not os.path.exists(new_flags_fn):
    with open(new_flags_fn, 'wb') as f:
        pickle.dump(new_flags, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Compressing the pickled file using the 
# Lempel–Ziv–Markov chain algorithm (LZMA) algorithm

new_flags_zx_fn = new_flags_fn.replace('pkl', 'xz')

if not os.path.exists(new_flags_zx_fn):
    with lzma.open(new_flags_zx_fn, 'wb') as f:
        pickle.dump(new_flags, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# # Reading back LZMA file
# with lzma.open(new_flags_zx_fn, 'rb') as file:
#     raw_data = file.read()
#     xz_flags = pickle.loads(raw_data)

## Reading LST-Binned file

In [ ]:
import numpy as np

from hera_cal.io import HERAData

from red_likelihood import group_data
from xd_utils import union_bad_ants

In [ ]:
test_uv = '/lustre/aoc/projects/hera/mmolnar/LST_bin/binned_files/zen.grp1.of1.LST.1.31552.HH.OCRSL.uvh5'
hd = HERAData(test_uv)

In [ ]:
data, flags, _ = hd.read(freq_chans=None, polarizations=['ee'])

In [ ]:
_, redg, cdata = group_data(test_uv, 'ee', chans=None, tints=None, bad_ants=union_bad_ants(idr2_jdsx))
flags_binned = cdata.mask